# Analysis of online laptop store
The goal of this analysis is to answer the following business questions:
- Given a laptop id, find the corresponding data.
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.
- Preprocess data to make queries run faster (improve time complexity) 

In [39]:
from csv import reader
with open('laptops.csv', mode = 'r', encoding = 'utf8') as file:
    #next(file)
    rows = list(reader(file))
    header, rows = rows[0], rows[1:] # assign header and body to different lists
print(rows[:5])

[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803']]


In [40]:
 """Define a class Inventory to read the file"""  
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, mode = 'r', encoding = 'utf8') as file:
            #next(file)
            rows = list(reader(file))
            for row in rows[1:]:
                row[-1] = int(row[-1])
            self.header, self.rows = rows[0], rows[1:]
            self.id_to_row = {}
            self.prices = set()
            def row_price(row):
                return row[-1]
            self.rows_by_price = sorted(self.rows, key = row_price)
            #print(self.rows_by_price)
            #pre-process self.rows into a dictionary
            for row in self.rows:
                self.id_to_row[row[0]]=row
            for row in self.rows:
                self.prices.add(int(row[-1]))
    def get_laptop_from_id(self, laptop_id):
        found = None
        for row in self.rows:
            if laptop_id == row[0]:
                found = row
        return found
        
    def get_laptop_from_id_fast(self, laptop_id):
        found = None
        if laptop_id in self.id_to_row.keys():
            found = self.id_to_row[laptop_id]
            return found
    def check_promotion_dollars(self, dollars):
        found = False
        for row in rows[:10]:
           # print(type(row[-1]))
            if int(row[-1]) == dollars:
                found = True
        for row1 in rows:
            for row2 in rows:
                if int(row1[-1]) + int(row2[-1]) == dollars:
                    found = True
        return found
    def check_promotion_dollars_fast(self, dollars):
        found = False
        if dollars in self.prices:
            found = True
        for row1 in self.rows:
            for row2 in self.rows:
                if int(row1[-1]) + int(row2[-1]) == dollars:
                    found = True
        return found
    def find_first_laptop_more_expensive(self, price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            mid_price = self.rows_by_price[range_middle][-1]
            if price == mid_price:                            
                return range_middle                    
            elif mid_price < price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        target_price = self.rows_by_price[range_start][-1]
        if price != target_price:                  
            return -1                                      
        return range_start + 1

"""Test the class constructor"""
laptops = Inventory('laptops.csv')
print(laptops.header)
print(len(laptops.rows))
print(type(laptops.rows[0][0]))
print(laptops.find_first_laptop_more_expensive(1000))
print(laptops.find_first_laptop_more_expensive(10000))



['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303
<class 'str'>
683
-1


In [41]:
"""Define a search function to locate a laptop information from Inventory.
   We need to update the inventory class with get_laptop_from_id_fast() method
"""
print(laptops.get_laptop_from_id('3362737'))
print("\n")
print(laptops.get_laptop_from_id('3362736'))
"""Test updated inventory class"""
print(laptops.get_laptop_from_id_fast('3362737'))
print("\n")
print(laptops.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


None
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


None


In [42]:
""" Time complexity test """
from time import time
from random import randint
ids = [str(randint(1000000,9999999)) for _ in range(10000)]
#print samples
print((ids[:5]))
# check with method is faster
total_time_no_dict = 0
for id in ids:
    start = time()
    laptops.get_laptop_from_id(id)
    end = time()
    total_time_no_dict += end-start
total_time_dict = 0
for id in ids:
    start = time()
    laptops.get_laptop_from_id_fast(id)
    end = time()
    total_time_dict += end-start
print('The times below \n')
print(total_time_no_dict)
print(total_time_dict)

['5094874', '8621029', '3041848', '5822800', '5355352']
The times below 

0.8351552486419678
0.0066606998443603516


"""
Update the Inventory class to include a method for gift cards spending,
call it check_promotion_dollars
"""

In [43]:
print(laptops.check_promotion_dollars(1000))
print(laptops.check_promotion_dollars(42))
print("\n")
print(laptops.check_promotion_dollars_fast(1000))
print(laptops.check_promotion_dollars_fast(42))

True
False


True
False


In [44]:
"""Performance Comparison""" 
prices = [randint(100, 5000) for _ in range(100)]


    

In [45]:
total_time_no_set = 0
for price in prices:
    start = time()
    laptops.check_promotion_dollars(price)
    total_time_no_set += time() - start
print(total_time_no_set)
    


87.31469488143921


In [46]:
total_time_set = 0
for price in prices:
    start = time()
    laptops.check_promotion_dollars_fast(price)
    total_time_set += time() - start
print(total_time_set)


60.44544982910156
